In [ ]:
import sys, os
import numpy as np
from evoman.environment import Environment
from demo_controller import player_controller

# Run test for 3 different enemies keeping the same parameters
experiment_name = 'dummy_demo'
solutions_dir = 'solutions'

if not os.path.exists(experiment_name):
    os.makedirs(experiment_name)

if not os.path.exists(solutions_dir):
    os.makedirs(solutions_dir)

crossover_parameter = 0.5
mutation_probability = 0.2
size_of_pop = 100
generations = 30
solutions_dir = 'solutions'
n_hidden = 10
dom_l = -1
dom_u = 1 

# Get fitness for individual
def simulate(env, individual):
    fitness, player_energy, enemy_energy, time = env.play(pcont=individual)
    return fitness

# Evaluate population
def evaluate(population, env):
    return np.array([simulate(env, individual) for individual in population])

# Select the fittest
def tourn_selection(population, fitness):
    index_1, index_2 = np.random.choice(np.arange(len(population)), size=2, replace=False)
    return population[index_1] if fitness[index_1] > fitness[index_2] else population[index_2]

# Uniform Crossover
def uniform_crossover(parent_1, parent_2):
    crossover_boolean = np.random.rand(len(parent_1)) < crossover_parameter
    offspring = np.where(crossover_boolean, parent_1, parent_2)  
    return offspring

# Mutation
def mutate(individual):
    mutation_positions = np.random.rand(len(individual)) < mutation_probability
    mutation_change = np.random.normal(0, 1, size=len(individual))
    individual[mutation_positions] = individual[mutation_positions] + mutation_change[mutation_positions]
    individual = np.clip(individual, dom_l, dom_u)
    return individual

# Initialize population
def initialize_population(size, number_of_variables):
    return np.random.uniform(-1, 1, (size, number_of_variables))

# Evolution
def evolution(env, size_of_pop, generations):
    amount_of_vars = (env.get_num_sensors() + 1) * n_hidden + (n_hidden + 1) * 5
    population = initialize_population(size_of_pop, amount_of_vars)
    fitness = evaluate(population, env)

    best_individuals = []  

    for generation in range(generations):
        new_pop = []

        for i in range(size_of_pop // 2):
            parent_1 = tourn_selection(population, fitness)
            parent_2 = tourn_selection(population, fitness)

            child1 = mutate(uniform_crossover(parent_1, parent_2))
            child2 = mutate(uniform_crossover(parent_2, parent_1))

            new_pop.extend([child1, child2])

        population = np.array(new_pop)
        fitness = evaluate(population, env)

        fittest_index = np.argmax(fitness)
        fittest_individual = population[fittest_index]
        best_individuals.append(fittest_individual) 
        std_fitness = np.std(fitness)

        print(f' From generation {generation} the fittest individual is {fitness[fittest_index]}, and the standard deviation is {std_fitness}')

        np.savetxt(f'{solutions_dir}/best_solution_gen_{generation}.txt', fittest_individual)
        with open(f'{experiment_name}/results.txt', 'a') as log_file:
            log_file.write(f'Generation {generation}: Best fitness = {fitness[fittest_index]} with standard deviation: {std_fitness}\n')

    return fittest_individual, best_individuals

# Main function
def run():
    all_best_individuals = [] 

    for enemy in [5]:
        print(f'\nResults for enemy {enemy}\n')

        # Initialize the environment for each enemy
        env = Environment(experiment_name=experiment_name,
                          enemies=[enemy],
                          playermode="ai",
                          player_controller=player_controller(n_hidden),
                          enemymode="static",
                          level=2,
                          speed="fastest",
                          visuals=False)

        print("Evolution has started")
        fittest_individual, best_individuals = evolution(env, size_of_pop, generations)
        all_best_individuals.append(fittest_individual)

        np.savetxt(f'{solutions_dir}/best_solution_final_enemy_{enemy}.txt', fittest_individual)
        print("Evolution is now finished. The final best solution is saved.")

    with open(f'{experiment_name}/all_best_individuals.txt', 'w') as f:
        for i, best_individual in enumerate(all_best_individuals):
            f.write(f'Best individual for enemy {5 + i}:\n{best_individual}\n\n')

run()